In [1]:
import pandas as pd
import difflib 
import re 

# --- 1. Definición de archivos y columnas ---
archivo_maestro = r"..\data\raw\hermosillo_colonias_poblacion.csv"
columna_maestra = "nom_col" # Columna con nombres correctos

archivo_a_corregir = r"..\data\raw\213.csv"
columna_a_corregir = "COLONIA" # Columna con nombres "sucios"

archivo_salida = r"..\data\raw\delitos.csv"

# --- 2. Configuración y Función de Normalización ---

SIMILARITY_THRESHOLD = 0.6

def normalize_name(name):
    """
    Limpia un nombre de colonia usando regex para una mejor comparación.
    """
    if not isinstance(name, str):
        return ""
    
    name = name.upper()
    name = name.replace('Á', 'A').replace('É', 'E').replace('Í', 'I').replace('Ó', 'O').replace('Ú', 'U')
    
    prefix_pattern = r'^(FRACCIONAMIENTO|FRACC|COLONIA|COL|RESIDENCIAL|RES|AMPLIACION|AMP|INVASION|INV|PRIVADA|PRIV)\.?\s+'
    name = re.sub(prefix_pattern, '', name, flags=re.IGNORECASE)
    
    name = re.sub(r'[^\w\s]', '', name)
    name = re.sub(r'\s+', ' ', name).strip()
    
    return name

# --- 3. Carga de Datos ---
try:
    df_poblacion = pd.read_csv(archivo_maestro)
    df_crimenes = pd.read_csv(archivo_a_corregir)
    print("Archivos cargados exitosamente.")
except Exception as e:
    print(f"Error al cargar los archivos: {e}")
    exit()

# --- 4. Preparación de Listas de Colonias 

try:
    clean_colonias_originales = df_poblacion[columna_maestra].dropna().astype(str).unique()
    print(f"Se encontraron {len(clean_colonias_originales)} colonias únicas en el archivo maestro.")
except KeyError:
    print(f"Error: No se encontró la columna '{columna_maestra}' en '{archivo_maestro}'.")
    exit()

print("Normalizando lista maestra con Regex...")
norm_to_clean_map = {}
for clean_name in clean_colonias_originales:
    norm_name = normalize_name(clean_name)
    if norm_name and norm_name not in norm_to_clean_map:
        norm_to_clean_map[norm_name] = clean_name

normalized_clean_list = list(norm_to_clean_map.keys())
print(f"Se generaron {len(normalized_clean_list)} nombres normalizados únicos para el mapeo.")

try:
    dirty_colonias_list = df_crimenes[columna_a_corregir].dropna().astype(str).str.upper().unique()
    print(f"Se encontraron {len(dirty_colonias_list)} colonias únicas en el archivo a corregir.")
except KeyError:
    print(f"Error: No se encontró la columna '{columna_a_corregir}' en '{archivo_a_corregir}'.")
    exit()


# --- 5. Lógica de Corrección
print(f"\nIniciando mapeo de corrección. Umbral de similitud = {SIMILARITY_THRESHOLD}...")

correction_map = {}
no_match_count = 0

for dirty_name in dirty_colonias_list: 
    normalized_dirty = normalize_name(dirty_name)
    
    if not normalized_dirty: 
        correction_map[dirty_name] = dirty_name 
        no_match_count += 1
        continue

    matches = difflib.get_close_matches(
        normalized_dirty,
        normalized_clean_list, 
        n=1,
        cutoff=SIMILARITY_THRESHOLD
    )

    if matches:
        best_normalized_match = matches[0] 
        best_clean_match = norm_to_clean_map[best_normalized_match]
        correction_map[dirty_name] = best_clean_match
    else:
        correction_map[dirty_name] = dirty_name
        no_match_count += 1

print(f"Se corregirán {len(dirty_colonias_list) - no_match_count} nombres de colonias.")
print(f"Se mantendrán {no_match_count} nombres de colonias sin cambios.")

# --- 6. Aplicación del Mapa y Guardado (MODIFICADO) ---

print("\nAplicando correcciones al DataFrame de crímenes...")

dirty_names_uppercase = df_crimenes[columna_a_corregir].astype(str).str.upper()

# Se sobrescribe la columna original (p.ej., 'COLONIA') en lugar de crear 'COLONIA_CORREGIDA'
df_crimenes[columna_a_corregir] = dirty_names_uppercase.map(correction_map)


try:
    # El DataFrame guardado ahora solo tendrá la columna 'COLONIA' corregida
    df_crimenes.to_csv(archivo_salida, index=False, encoding='utf-8-sig')
    print(f"\n¡Proceso completado! Se ha guardado el archivo corregido como: '{archivo_salida}'")
except Exception as e:
    print(f"\nError al guardar el archivo: {e}")


# --- 7. Guardar el Mapeo de Corrección ---
print("\nGuardando el diccionario de mapeo en un CSV...")

# Define el nombre del archivo de mapeo
archivo_mapeo_salida = r"..\data\raw\mapeo_correccion_colonias.csv"

try:
    # Convierte el diccionario 'correction_map' a un DataFrame
    df_mapeo = pd.DataFrame(
        list(correction_map.items()), 
        columns=['COLONIA_ORIGINAL_SUCIA', 'COLONIA_CORREGIDA_MAESTRA']
    )
    
    # Ordena para que sea más fácil de revisar
    df_mapeo = df_mapeo.sort_values(by=['COLONIA_CORREGIDA_MAESTRA', 'COLONIA_ORIGINAL_SUCIA'])
    
    # Guarda el DataFrame de mapeo en un CSV
    df_mapeo.to_csv(archivo_mapeo_salida, index=False, encoding='utf-8-sig')
    
    print(f"¡Éxito! Mapeo guardado en: '{archivo_mapeo_salida}'")

except Exception as e:
    print(f"\nError al guardar el archivo de mapeo: {e}")

Archivos cargados exitosamente.
Se encontraron 660 colonias únicas en el archivo maestro.
Normalizando lista maestra con Regex...
Se generaron 655 nombres normalizados únicos para el mapeo.
Se encontraron 1407 colonias únicas en el archivo a corregir.

Iniciando mapeo de corrección. Umbral de similitud = 0.6...
Se corregirán 1286 nombres de colonias.
Se mantendrán 121 nombres de colonias sin cambios.

Aplicando correcciones al DataFrame de crímenes...

¡Proceso completado! Se ha guardado el archivo corregido como: '..\data\raw\delitos.csv'

Guardando el diccionario de mapeo en un CSV...
¡Éxito! Mapeo guardado en: '..\data\raw\mapeo_correccion_colonias.csv'


In [4]:
#sacamos las colonias unicas corregidas
df_crimenes['COLONIA'].nunique()

678

In [ ]:
import pandas as pd
import googlemaps
import time

# --- 1. Configuración ---
# ¡NUNCA compartas esta clave! Pégala aquí.
SU_API_KEY = "PEGA_TU_API_KEY_DE_GOOGLE_AQUI"

ARCHIVO_COLONIAS = "hermosillo_colonias_poblacion.csv"
COLUMNA_NOMBRES = "nom_col"
CIUDAD_CONTEXTO = ", Hermosillo, Sonora"

# --- 2. Inicializar el cliente de Google Maps ---
try:
    gmaps = googlemaps.Client(key=SU_API_KEY)
except ValueError as e:
    print(f"Error al inicializar el cliente de Google Maps: {e}")
    print("Por favor, asegúrate de que tu API_KEY es correcta y está entre comillas.")
    exit()

print(f"Cargando archivo: {ARCHIVO_COLONIAS}")
try:
    df = pd.read_csv(ARCHIVO_COLONIAS)
except FileNotFoundError:
    print(f"Error: No se encontró el archivo '{ARCHIVO_COLONIAS}'")
    exit()
except Exception as e:
    print(f"Error al leer el archivo CSV: {e}")
    exit()

# Asegurarnos de que las columnas de lat/lon no existan ya
if 'latitud' not in df.columns:
    df['latitud'] = pd.NA
if 'longitud' not in df.columns:
    df['longitud'] = pd.NA

# --- 3. Obtener la lista de colonias únicas que faltan por geocodificar ---
# (Esto ahorra muchísimas llamadas a la API si el script se interrumpe)
colonias_unicas = df[df['latitud'].isna()][COLUMNA_NOMBRES].dropna().unique()

print(f"Se encontraron {len(colonias_unicas)} colonias únicas para geocodificar.")

# --- 4. Iterar y Geocodificar ---
geocode_map = {} # Un diccionario para guardar los resultados temporalmente

for i, nombre_colonia in enumerate(colonias_unicas):
    
    # Formateamos la búsqueda como pediste
    query_busqueda = nombre_colonia + CIUDAD_CONTEXTO
    
    print(f"Buscando ({i+1}/{len(colonias_unicas)}): '{query_busqueda}'...")
    
    try:
        # Llama a la API de Geocoding
        geocode_result = gmaps.geocode(query_busqueda)
        
        if geocode_result:
            # Extrae la latitud y longitud del primer resultado
            location = geocode_result[0]['geometry']['location']
            lat = location['lat']
            lng = location['lng']
            
            geocode_map[nombre_colonia] = (lat, lng)
            print(f"  -> Encontrado: ({lat}, {lng})")
        else:
            print("  -> No se encontraron resultados.")
            geocode_map[nombre_colonia] = (None, None)
            
    except Exception as e:
        print(f"  -> ERROR durante la solicitud: {e}")
        geocode_map[nombre_colonia] = (None, None)
    
    # IMPORTANTE: Pausa breve para no saturar la API y evitar errores "RATE_LIMIT"
    time.sleep(0.05) # 50 milisegundos

# --- 5. Aplicar los resultados al DataFrame ---
print("\nGeocodificación completada. Aplicando resultados al DataFrame...")

# Creamos funciones map temporales para latitud y longitud
def get_lat(colonia):
    return geocode_map.get(colonia, (pd.NA, pd.NA))[0]

def get_lon(colonia):
    return geocode_map.get(colonia, (pd.NA, pd.NA))[1]

# Aplicamos el mapeo SÓLO a las filas que tenían NaN
df['latitud'] = df['latitud'].fillna(df[COLUMNA_NOMBRES].apply(get_lat))
df['longitud'] = df['longitud'].fillna(df[COLUMNA_NOMBRES].apply(get_lon))

# --- 6. Guardar el archivo ---
try:
    # Guardamos en el mismo archivo, sobrescribiéndolo
    df.to_csv(ARCHIVO_COLONIAS, index=False, encoding='utf-8-sig')
    print(f"\n¡Éxito! Archivo actualizado y guardado como: '{ARCHIVO_COLONIAS}'")
except Exception as e:
    print(f"\nError al guardar el archivo: {e}")

1253